# Tuning Script

In [1]:
# Parameters subject to change in every script - involved in naming

MODEL = 'svc(rbf)'
TUNER = 'yangzhoub'

---

In [2]:
# detect whether this is a google environment

COLAB_ENVIRONMENT = False

try:
    from google.colab import drive
    drive.mount('/content/drive')
    COLAB_ENVIRONMENT = True
except:
    pass

Mounted at /content/drive


In [3]:
import sys
import os
import pickle

if COLAB_ENVIRONMENT:
    home_directory = './drive/MyDrive/LAB/AFLBrownlowPredictor2024/' # my home directory is stored in ./LAB of google drive
    if MODEL == 'ebr':
        !pip install interpret==0.5.0
    elif MODEL == 'cbr':
        !pip install catboost
    !pip install JXAutoML
else:
    home_directory = '../../'

from JXAutoML.YangZhouB import YangZhouB as Tuner

import pandas as pd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 1.5 MB/s eta 0:00:00


In [4]:
# Parameters that are same for every script
SEED = 42

TASK_TYPE = 'Regression'  # this task is a classification task

# we have decided to tune features as hp
TUNE_FEATURES_AS_HYPERPARAMETERS = True

---

In [5]:
# model to use to tune
from sklearn.svm import SVR as clf

# what values to try for each hyperparameter
parameter_choices = {
    'C': (0.0001, 0.001, 0.01, 0.1, 1, 10),
    'tol': (0.0001, 0.001, 0.01, 0.1, 1, 10),
    'max_iter': (50, 100, 200, 400, 800, 1600),
    'gamma': ('scale', 'auto')
}

# what values to set non-tuneable parameters/hyperparameters
non_tunable_hyperparameters_dict = {
                                    'verbose': False,
                                    }

---

Read in and Prepare Data

In [6]:
train_data = pd.read_csv(f'{home_directory}data/curated/modelling/train.csv')
val_data = pd.read_csv(f'{home_directory}data/curated/modelling/val.csv')
test_data = pd.read_csv(f'{home_directory}data/curated/modelling/test.csv')

train_X = train_data.drop(columns=['target'])
train_y = train_data['target']
val_X = val_data.drop(columns=['target'])
val_y = val_data['target']
test_X = test_data.drop(columns=['target'])
test_y = test_data['target']

In [7]:
# Import feature importance ordering

with open(f'{home_directory}models/feature_importance_ordering.pickle', 'rb') as f:
    feature_importance_ordering = pickle.load(f)

Set parameters

In [8]:
# initialisation
tuner = Tuner()

print('---')

# define what model we are tuning
tuner.read_in_model(clf, TASK_TYPE, pytorch_model=False, optimised_metric='r2')

print('---')

# read in the data for training and validation
tuner.read_in_data(train_X, train_y, val_X, val_y, test_X, test_y)

print('---')

# set what hp values to tune
tuner.set_hyperparameters(parameter_choices)
# WARNING: this may take a while if no. tuneable hyperparameters are large

print('---')

# set up hp values that need to be changed from default but NOT to be tuned
tuner.set_non_tuneable_hyperparameters(non_tunable_hyperparameters_dict)

print('---')

# set up feature importance ordering

if TUNE_FEATURES_AS_HYPERPARAMETERS:
    tuner.set_features(feature_importance_ordering)
    # WARNING: this may take a while if no. tuneable hyperparameters are large

print('---')

try:  # try to read in previous results to continue tuning ...
    tuner.read_in_tuning_result_df(
        f'{home_directory}models/tuning/{TUNER}_{MODEL}.csv')
except:
    print('No previous tuning data read in')

print('---')

# set up where to save the tuning result csv
tuner.set_tuning_result_saving_address(
    f'{home_directory}models/tuning/{TUNER}_{MODEL}.csv')

print('---')

# set up where to save the current best model
tuner.set_best_model_saving_address(
    f'{home_directory}models/tmp_models/{TUNER}_{MODEL}')

YangZhouB Initialised
---
Successfully read in model <class 'sklearn.svm._classes.SVR'>, which is a Regression model optimising for r2
---
Read in Train X data
Read in Train y data
Read in Val X data
Read in Val y data
Read in Test X data
Read in Test y data
---
Successfully recorded hyperparameter choices
---
Successfully recorded non_tuneable_hyperparameter choices
---
Successfully recorded tuneable feature combination choices and updated relevant internal structures
---
---
Successfully set tuning output address
---
Successfully set best model output address


In [9]:
import warnings
warnings.filterwarnings("ignore")

In [10]:
# begin tuning ... (or continue training)

tuner.tune()

BEGIN TUNING


STAGE ZERO: Tune all Cruise combinations


Trained and Tested combination 1 of 15984: (5, 5, 5, 1, 10), taking 0.17 seconds to get val score of 0.0
        Current best combo: (5, 5, 5, 1, 10) with val score 0.0
Trained and Tested combination 2 of 15984: (0, 0, 5, 0, 0), taking 14.05 seconds to get val score of 0.0472
        Current best combo: (0, 0, 5, 0, 0) with val score 0.0472
Trained and Tested combination 3 of 15984: (5, 0, 0, 1, 0), taking 0.43 seconds to get val score of -0.6235
        Current best combo: (0, 0, 5, 0, 0) with val score 0.0472
Trained and Tested combination 4 of 15984: (0, 5, 0, 1, 28), taking 0.03 seconds to get val score of 0.0
        Current best combo: (0, 0, 5, 0, 0) with val score 0.0472
Trained and Tested combination 5 of 15984: (5, 0, 0, 1, 32), taking 0.33 seconds to get val score of 0.6517
        Current best combo: (5, 0, 0, 1, 32) with val score 0.6517
Trained and Tested combination 6 of 15984: (5, 5, 0, 0, 20), taking 0.03 second